In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
df = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')
df.head()

In [ ]:
df.shape

# Preprocessing - convert to numeric data

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
le = LabelEncoder()

In [ ]:
ds = df.apply(func=le.fit_transform) # this will be applied to each column : meaning of apply
ds.head()

In [ ]:
data = ds.values # becomes a list
data

In [ ]:
X = data[:, 1:] # all colsexcept 1st
y = data[:, 0] # all rows, just 0 col
X.shape, y.shape

# Custom Naive Bayes

In [ ]:
class CustomNB:
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    # label = which class you want this prob for
    def prior_prob(self, label): 
        total = self.y_train.shape[0]
        class_examples = np.sum(self.y_train == label)
        return class_examples / float(total)  # python 2
    
    # P(Xi=red|y=label) - ith feature (feature col = i) for a single example
    def conditional_prob(self, feature_col, feature_val, label):
        # out of all the examples, what mushrooms have feature as feature_val in the feature_col that belongs to that class label
        X_filtered = self.X_train[self.y_train==label] # all the examples in class label
        numerator = np.sum(X_filtered[:, feature_col] == feature_val)
        denominator = len(X_filtered)
        return numerator / denominator
    
    # we are going to do this for all the 22 features that we have for each example
    def predict_point(self, X_test):
        # X_test is a single example with n features
        classes = np.unique(self.y_train) # By default from 0
        n_features = self.X_train.shape[1]
        post_pro = []
        # post prob for each class
        for label in classes:
            # post_prob = prior * likelihood
            likehood = 1.0
            for feature in range(n_features):
                cond = self.conditional_prob(feature, X_test[feature], label)
                likehood *= cond
            prior = self.prior_prob(label)
            post = prior * likehood
            post_pro.append(post)
        
        # ans = max value from all labels
        return np.argmax(post_pro) # return the index of the largest value in array
    
    def predict(self, X_test):
        result = []
        for point in X_test:
            result.append(self.predict_point(point))
        return np.array(result)
    
    def score(self, X_test, y_test):
        return (self.predict(X_test) == y_test).mean()

### Splitting Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = CustomNB()
model.fit(X_train, y_train)

In [ ]:
model.predict(X_test[:10])

In [ ]:
y_test[:10]

# Accuracy

In [ ]:
ans = model.score(X_test, y_test)
print('Accuracy of the custom Naive Bayes model is :',ans * 100)